In [ ]:
pip install biopython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 8.8 MB/s eta 0:00:00


In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
file = open('ids.txt', 'r')
ids = file.readlines()

pmids = []
for pmc in ids:
  pmid = pmc.strip()
  pmids.append(pmid)

len(pmids)


10000

In [ ]:
pmids = pmids[:2000]

In [ ]:
import time
import pandas as pd
import string
from Bio import Entrez

# Function to fetch paper details
def fetch_paper_details(pmid):
    handle = Entrez.efetch(db="pubmed", id=pmid)
    record = Entrez.read(handle)
    handle.close()
    return record

# Initialize empty lists to store data
titles = []
abstracts = []
author_keywords = []

# Assuming pmids is a list of PubMed IDs
for pmid in pmids:
    record = fetch_paper_details(pmid)

    try:
        paper_title = record['PubmedArticle'][0]['MedlineCitation']['Article']['ArticleTitle']

        paper_abstract = ''
        abstract_record = record['PubmedArticle'][0]['MedlineCitation']['Article']['Abstract']['AbstractText']
        abs_n = len(abstract_record)
        for i in range(abs_n):
            paper_abstract += abstract_record[i]

        keywords_list = record['PubmedArticle'][0]['MedlineCitation']['KeywordList'][0]
        paper_keywords = ''
        key_n = len(keywords_list)
        for i in range(key_n):
          paper_keywords += f'{keywords_list[i]},'

        paper_keywords_list = paper_keywords.split(',')
        paper_keywords_list = [keyword.strip().translate(str.maketrans('', '', string.punctuation)) for keyword in paper_keywords_list if keyword.strip()]

        if paper_title and paper_abstract and paper_keywords_list:
          titles.append(paper_title)
          abstracts.append(paper_abstract)
          author_keywords.append(paper_keywords_list)

    except Exception as e:
        continue

    time.sleep(1)

In [ ]:
data = {'title': titles, 'abstract': abstracts, 'author_keywords': author_keywords}
df = pd.DataFrame(data)

df.head()

,title,abstract,author_keywords
0,Exercise and other non-pharmaceutical interven...,To assess the relative effects of different ty...,"[cancer related fatigue, exercise, indirect co..."
1,Effectiveness of conservative interventions in...,To investigate the effectiveness of conservati...,"[Conservative, MetaAnalysis, Pain, Shoulder Im..."
2,Effectiveness of myopia control interventions:...,This study aims to investigate the effectivene...,"[axial length, efficacy, elongation, myopia, p..."
3,Causal relationships between gut microbiota an...,Multiple clinical studies have indicated that ...,"[PD1PDL1, bidirectional Mendelian randomizatio..."
4,Prevention of hypotension after spinal anaesth...,Spinal anaesthesia for caesarean section commo...,"[anaesthesia, spinal, caesarean section, hypot..."


In [ ]:
print(df.loc[0, 'title'], "\n")
print(df.loc[0, 'abstract'], "\n")
print(df.loc[0, 'author_keywords'])

Exercise and other non-pharmaceutical interventions for cancer-related fatigue in patients during or after cancer treatment: a systematic review incorporating an indirect-comparisons meta-analysis. 

To assess the relative effects of different types of exercise and other non-pharmaceutical interventions on cancer-related fatigue (CRF) in patients during and after cancer treatment.Systematic review and indirect-comparisons meta-analysis.Articles were searched in PubMed, Cochrane CENTRAL and published meta-analyses.Randomised studies published up to January 2017 evaluating different types of exercise or other non-pharmaceutical interventions to reduce CRF in any cancer type during or after treatment.Risk of bias assessment with PEDro criteria and random effects Bayesian network meta-analysis.We included 245 studies. Comparing the treatments with usual care during cancer treatment, relaxation exercise was the highest ranked intervention with a standardisedmean difference (SMD) of -0.77 (9

In [ ]:
df.shape

(1193, 3)

In [ ]:
df.to_csv('pubmed_papers.csv', index=False)